In [1]:
import pandas as pd

In [2]:
filename = '01Feb2021/NIFTY21MARFUT_1.CSV'
orderdf = pd.read_csv(filename)

In [13]:
orderdf = pd.read_csv(filename)
orderdf.columns= cols = ['Timestamp', 'TIME', 'Ord_ID1', 'Ord_ID2', 'Price', 'Qty',\
       'MessageType', 'LTP', 'LTQ', 'VTT', 'ORDER TYPE', 'BIDQ1', 'BID1',\
       'ASK1', 'ASKQ1', 'BIDQ2', 'BID2', 'ASK2', 'ASKQ2', 'BIDQ3', 'BID3',\
       'ASK3', 'ASKQ3', 'BIDQ4', 'BID4', 'ASK4', 'ASKQ4', 'BIDQ5', 'BID5',\
       'ASK5', 'ASKQ5', 'last']
orderdf.index.name = 'Sqn'

orderdf['TIME'] = pd.to_datetime(orderdf['TIME'], format='%H:%M:%S.%f')
orderdf = orderdf.set_index(['TIME'])
orderdf.head()

,Timestamp,Ord_ID1,Ord_ID2,Price,Qty,MessageType,LTP,LTQ,VTT,ORDER TYPE,...,ASKQ3,BIDQ4,BID4,ASK4,ASKQ4,BIDQ5,BID5,ASK5,ASKQ5,last
TIME,,,,,,,,,,,,,,,,,,,,,
1900-01-01 09:15:00.009974,1296638100009974214,1100000000000913,0,13725.10,3000,NEW ORDER,0.0,0,0,BUY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1900-01-01 09:15:00.017551,1296638100017551794,1100000000001311,0,14135.35,75,NEW ORDER,0.0,0,0,SELL,...,75.0,NaN,NaN,14299.45,225.0,NaN,NaN,NaN,NaN,NaN
1900-01-01 09:15:00.017624,1296638100017624413,1100000000001319,0,13725.15,150,NEW ORDER,0.0,0,0,BUY,...,75.0,600.0,13501.5,14299.45,225.0,NaN,NaN,NaN,NaN,NaN
1900-01-01 09:15:00.017632,1296638100017632126,1100000000001320,0,13904.85,150,NEW ORDER,0.0,0,0,SELL,...,600.0,600.0,13501.5,14135.35,75.0,NaN,NaN,14299.45,225.0,NaN
1900-01-01 09:15:00.017645,1296638100017645558,1100000000001323,0,13904.85,225,NEW ORDER,0.0,0,0,SELL,...,600.0,600.0,13501.5,14135.35,75.0,NaN,NaN,14299.45,225.0,NaN


In [49]:
def dataprocessing(filename):
    orderdf = pd.read_csv(filename)
    orderdf.columns = cols
    orderdf.index.name = 'Sqn'
    orderdf['TIME'] = pd.to_datetime(orderdf['TIME'], format='%H:%M:%S.%f')
    orderdf = orderdf.set_index(['TIME'])
    price_ohlc = orderdf['Price'].resample('60s').ohlc()
    tradedf = orderdf[orderdf.MessageType == 'TRADE']
    successful_trades = tradedf['MessageType'].resample('1min').count()
    cancelled_trades = orderdf[orderdf.MessageType == 'CANCELLATION']['MessageType'].resample('1min').count()
    new_orders = orderdf[orderdf.MessageType == 'NEW ORDER']['MessageType'].resample('1min').count()
    modified_df = orderdf[orderdf.MessageType == 'MODIFICATION']['MessageType'].resample('1min').count()
    
    df = pd.DataFrame()
    df = pd.concat([df, new_orders, cancelled_trades, successful_trades, modified_df], axis=1)
    df.columns = ['neworders', 'cancelled_orders', 'trades', 'modified']

    return orderdf, df

In [50]:
orderdf, new_df = dataprocessing(filename)

In [51]:
orderdf

,Timestamp,Ord_ID1,Ord_ID2,Price,Qty,MessageType,LTP,LTQ,VTT,ORDER TYPE,...,ASKQ3,BIDQ4,BID4,ASK4,ASKQ4,BIDQ5,BID5,ASK5,ASKQ5,last
TIME,,,,,,,,,,,,,,,,,,,,,
1900-01-01 09:15:00.009974,1296638100009974214,1100000000000913,0,13725.10,3000,NEW ORDER,0.00,0,0,BUY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1900-01-01 09:15:00.017551,1296638100017551794,1100000000001311,0,14135.35,75,NEW ORDER,0.00,0,0,SELL,...,75.0,NaN,NaN,14299.45,225.0,NaN,NaN,NaN,NaN,NaN
1900-01-01 09:15:00.017624,1296638100017624413,1100000000001319,0,13725.15,150,NEW ORDER,0.00,0,0,BUY,...,75.0,600.0,13501.5,14299.45,225.0,NaN,NaN,NaN,NaN,NaN
1900-01-01 09:15:00.017632,1296638100017632126,1100000000001320,0,13904.85,150,NEW ORDER,0.00,0,0,SELL,...,600.0,600.0,13501.5,14135.35,75.0,NaN,NaN,14299.45,225.0,NaN
1900-01-01 09:15:00.017645,1296638100017645558,1100000000001323,0,13904.85,225,NEW ORDER,0.00,0,0,SELL,...,600.0,600.0,13501.5,14135.35,75.0,NaN,NaN,14299.45,225.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1900-01-01 09:29:59.915602,1296638999915602174,1100000002819220,0,13774.70,75,CANCELLATION,13773.95,75,93225,SELL,...,75.0,150.0,13765.5,13775.00,375.0,75.0,13765.0,13775.95,150.0,NaN
1900-01-01 09:29:59.915605,1296638999915605114,1100000002819221,0,13774.80,75,CANCELLATION,13773.95,75,93225,SELL,...,375.0,150.0,13765.5,13775.95,150.0,75.0,13765.0,13776.00,75.0,NaN
1900-01-01 09:29:59.961084,1296638999961084147,1100000002819656,0,13773.20,75,NEW ORDER,13773.95,75,93225,SELL,...,150.0,150.0,13765.5,13775.00,375.0,75.0,13765.0,13775.95,150.0,NaN


In [52]:
order_df_traded = orderdf[orderdf['MessageType'] == 'TRADE']

In [53]:
market_orders = order_df_traded[((order_df_traded['ORDER TYPE'] == 'BUY') & (order_df_traded['Price'] >= order_df_traded['ASK1'])) \
                    | ((order_df_traded['ORDER TYPE'] == 'SELL') & (order_df_traded['Price'] <= order_df_traded['BID1']))]

market_orders_df = market_orders['ORDER TYPE'].resample('1min').count()

**Observation** I think cancelled and rejected orders should be ignored. Market behaviour, it's aggresiveness can be understood better by the real orders. So, it's better to have a look at actually traded orders and modified orders. 

Even for modified orders, let's keep it as a hypothesis to see their influence on market movement.


- I am missing to consider sl orders. Will have to find out a way to check them. If sl executes then it can be of 2 types:
   - SLM : Market order but SL
   - SLL 

In [84]:
## Traded clip sizes

avg_clip    = order_df_traded['Qty'].resample('1min').mean() 
median_clip = order_df_traded['Qty'].resample('1min').median()
min_clip    = order_df_traded['Qty'].resample('1min').min()
max_clip    = order_df_traded['Qty'].resample('1min').max()
clipsize = pd.DataFrame({'avg_clip': avg_clip, 'median_clip': median_clip,
                        'min_clip': min_clip, 'max_clip': max_clip})                        

In [85]:
price_ohlc = order_df_traded['Price'].resample('60s').ohlc()

In [87]:
#market_order_clipsize 
import numpy as np
mo_clip_mean = market_orders['Qty'].resample('1min').mean().replace(np.NaN, 0)
mo_clip_median = market_orders['Qty'].resample('1min').median().replace(np.NaN, 0)
mo_clip_min = market_orders['Qty'].resample('1min').min().replace(np.NaN, 0)
mo_clip_max  =  market_orders['Qty'].resample('1min').max().replace(np.NaN, 0)
mo_clipsize = pd.DataFrame({'mo_avg_clip': mo_clip_mean, 'mo_median_clip': mo_clip_median,
                        'mo_min_clip': mo_clip_min, 'mo_max_clip': mo_clip_max})

In [101]:
df = pd.DataFrame()
df = pd.concat([df, price_ohlc, new_df, market_orders_df, clipsize, mo_clipsize], axis=1)

In [102]:
df = df.rename(columns = {'ORDER TYPE':'market_orders'})

In [104]:
df

,open,high,low,close,neworders,cancelled_orders,trades,modified,market_orders,avg_clip,median_clip,min_clip,max_clip,mo_avg_clip,mo_median_clip,mo_min_clip,mo_max_clip
1900-01-01 09:15:00,13824.95,13826.15,13773.95,13775.40,2823,2656,238,35543,31,138.340336,75,75,1350,120.967742,75.0,75.0,375.0
1900-01-01 09:16:00,13787.50,13798.70,13780.70,13794.80,2201,2180,77,32260,4,111.038961,75,75,825,112.500000,75.0,75.0,225.0
1900-01-01 09:17:00,13794.85,13806.15,13790.90,13804.00,1467,1392,105,26411,9,105.000000,75,75,375,100.000000,75.0,75.0,150.0
1900-01-01 09:18:00,13805.00,13811.50,13799.90,13806.55,2120,2017,77,36256,5,92.532468,75,75,375,75.000000,75.0,75.0,75.0
1900-01-01 09:19:00,13807.90,13808.80,13784.35,13786.20,2044,1957,66,38035,8,84.090909,75,75,225,75.000000,75.0,75.0,75.0
1900-01-01 09:20:00,13780.90,13780.90,13747.05,13770.75,4501,4498,59,53888,9,97.881356,75,75,375,91.666667,75.0,75.0,150.0
1900-01-01 09:21:00,13763.80,13770.00,13750.00,13765.20,2637,2662,37,35269,4,99.324324,75,75,225,112.500000,75.0,75.0,225.0
1900-01-01 09:22:00,13768.80,13791.00,13764.70,13776.10,3503,3378,24,39836,2,75.000000,75,75,75,75.000000,75.0,75.0,75.0
1900-01-01 09:23:00,13780.00,13799.40,13780.00,13790.25,3472,3369,31,39849,1,82.258065,75,75,150,75.000000,75.0,75.0,75.0
1900-01-01 09:24:00,13790.25,13790.60,13780.95,13781.25,3006,2917,9,37478,0,83.333333,75,75,150,0.000000,0.0,0.0,0.0


In [108]:
df.columns

Index(['open', 'high', 'low', 'close', 'neworders', 'cancelled_orders',
       'trades', 'modified', 'market_orders', 'avg_clip', 'median_clip',
       'min_clip', 'max_clip', 'mo_avg_clip', 'mo_median_clip', 'mo_min_clip',
       'mo_max_clip', 'market_aggressive', 'up_trend', 'fill_rate'],
      dtype='object')

- Market Aggressiveness
- Up/Down move in last 1 min
- fill rate
- plot chart

In [106]:
df['market_aggressive'] = df.market_orders /df.trades
df['up_trend']          = (df.close - df.open) >= 0
df['fill_rate']         = df.trades/df.neworders

In [124]:
def prep_data(orderdf, df):
    order_df_traded = orderdf[orderdf['MessageType'] == 'TRADE']
    market_orders = order_df_traded[((order_df_traded['ORDER TYPE'] == 'BUY') & (order_df_traded['Price'] >= order_df_traded['ASK1'])) \
                    | ((order_df_traded['ORDER TYPE'] == 'SELL') & (order_df_traded['Price'] <= order_df_traded['BID1']))]
    price_ohlc = order_df_traded['Price'].resample('60s').ohlc()
    df = pd.concat([df, price_ohlc], axis=1)
    df['market_orders'] = market_orders['ORDER TYPE'].resample('1min').count()
    df['mo_avg_clip'] = market_orders['Qty'].resample('1min').mean().replace(np.NaN, 0)
    df['mo_median_clip'] = market_orders['Qty'].resample('1min').median().replace(np.NaN, 0)
    df['mo_min_clip'] = market_orders['Qty'].resample('1min').min().replace(np.NaN, 0)
    df['mo_max_clip']  =  market_orders['Qty'].resample('1min').max().replace(np.NaN, 0)
    ## Traded clip sizes

    df['avg_clip ']   = order_df_traded['Qty'].resample('1min').mean() 
    df['median_clip'] = order_df_traded['Qty'].resample('1min').median()
    df['min_clip']    = order_df_traded['Qty'].resample('1min').min()
    df['max_clip']    = order_df_traded['Qty'].resample('1min').max()
    df['market_aggressive'] = df.market_orders /df.trades
    df['up_trend']          = (df.close - df.open) >= 0
    df['fill_rate']         = df.trades/df.neworders
    print("Done")
    return df

In [125]:
import pandas as pd
import os
import natsort 
import re

files = natsort.natsorted(os.listdir('01Feb2021/'))

feb, mar, apr = [], [], []
contracts = [feb, mar, apr]
for i in files:
    if re.search('FEB', i):
        feb.append(i)
    elif re.search('MAR', i):
        mar.append(i)
    elif re.search('APR', i):
        apr.append(i)
        
os.chdir('./01Feb2021')

FileNotFoundError: [WinError 3] The system cannot find the path specified: '01Feb2021/'

In [126]:
feb_contract_df = pd.DataFrame()

In [127]:
for i in feb:
    data, df = dataprocessing(i)
    df = prep_data(data, df)
    feb_contract_df = pd.concat([feb_contract_df, df], axis = 0)

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


In [128]:
feb_contract_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 388 entries, 1900-01-01 09:15:00 to 1900-01-01 15:29:00
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   neworders          378 non-null    float64
 1   cancelled_orders   377 non-null    float64
 2   trades             376 non-null    float64
 3   modified           382 non-null    float64
 4   open               376 non-null    float64
 5   high               376 non-null    float64
 6   low                376 non-null    float64
 7   close              376 non-null    float64
 8   market_orders      375 non-null    float64
 9   mo_avg_clip        375 non-null    float64
 10  mo_median_clip     375 non-null    float64
 11  mo_min_clip        375 non-null    float64
 12  mo_max_clip        375 non-null    float64
 13  avg_clip           376 non-null    float64
 14  median_clip        376 non-null    float64
 15  min_clip           376 non-null    fl

### Analysis
Check when market aggressiveness > 0.25 

In [131]:
feb_contract_df[feb_contract_df.market_aggressive > 0.25]

,neworders,cancelled_orders,trades,modified,open,high,low,close,market_orders,mo_avg_clip,mo_median_clip,mo_min_clip,mo_max_clip,avg_clip,median_clip,min_clip,max_clip,market_aggressive,up_trend,fill_rate
1900-01-01 10:08:00,1648.0,1204.0,796.0,3156.0,13774.40,13794.00,13773.00,13790.10,225.0,95.666667,75.0,75.0,750.0,118.718593,75.0,75.0,1950.0,0.282663,True,0.483010
1900-01-01 10:40:00,1474.0,1007.0,924.0,2961.0,13782.65,13807.00,13782.65,13804.75,280.0,94.821429,75.0,75.0,600.0,106.168831,75.0,75.0,1575.0,0.303030,True,0.626866
1900-01-01 10:44:00,1569.0,1210.0,510.0,2999.0,13796.15,13810.00,13794.00,13808.95,132.0,113.068182,75.0,75.0,900.0,146.176471,75.0,75.0,3450.0,0.258824,True,0.325048
1900-01-01 11:30:00,4461.0,3157.0,1420.0,22958.0,13891.15,13905.00,13888.20,13897.35,370.0,107.432432,75.0,75.0,1050.0,116.725352,75.0,75.0,2100.0,0.260563,True,0.318314
1900-01-01 11:45:00,1780.0,1318.0,589.0,4482.0,13883.95,13900.00,13881.60,13897.00,151.0,91.390728,75.0,75.0,1050.0,94.736842,75.0,75.0,1500.0,0.256367,True,0.330899
1900-01-01 12:50:00,4707.0,3318.0,2626.0,8574.0,13896.50,13965.85,13889.50,13958.70,728.0,104.464286,75.0,75.0,2025.0,114.384996,75.0,75.0,4725.0,0.277228,True,0.557893


In [181]:
feb_contract_df.columns

Index(['neworders', 'cancelled_orders', 'trades', 'modified', 'open', 'high',
       'low', 'close', 'market_orders', 'mo_avg_clip', 'mo_median_clip',
       'mo_min_clip', 'mo_max_clip', 'avg_clip ', 'median_clip', 'min_clip',
       'max_clip', 'market_aggressive', 'up_trend', 'fill_rate'],
      dtype='object')

In [134]:
import datetime

In [179]:
feb_contract_df = feb_contract_df.dropna()
feb_contract_df.index = pd.date_range("2021-02-01 09:15:00", "2021-02-01 15:29:00", freq="1min")

In [180]:
import plotly.graph_objects as go
fig = go.Figure([go.Scatter(x=feb_contract_df.index, y=feb_contract_df.close)])
fig.show()


In [191]:
log_price = np.log(feb_contract_df.close) - 9

In [192]:
import plotly.express as px

fig = px.line(feb_contract_df, x=feb_contract_df.index,\
              y=[ feb_contract_df.market_aggressive, feb_contract_df.fill_rate, log_price],
              title='custom tick labels')
fig.show()
